In [6]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

# Load the movie dataset
data_path = "custom_movie_dataset.csv"  # Update with the actual path to your data file
movies = pd.read_csv('/content/sample_data/custom_movie_dataset.csv')

# Merge key attributes like genre and director into a single string for each movie
movies["combined_features"] = movies["genre"] + " " + movies["director"]

# Transform the combined features into a frequency count matrix
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(movies["combined_features"])

# Calculate the cosine similarity matrix
similarity_matrix = cosine_similarity(count_matrix)

# Function to suggest movies based on the genre
def suggest_movies_by_genre(input_genre, top_n_recommendations=5):
    # Select movies whose genre matches the input
    genre_filtered_movies = movies[movies["genre"].str.contains(input_genre, case=False, na=False)]

    if genre_filtered_movies.empty:
        return f"No movies found under the genre '{input_genre}'."

    # For the selected genre movies, calculate similarity scores
    genre_movie_indices = genre_filtered_movies.index
    all_similarity_scores = []

    for index in genre_movie_indices:
        all_similarity_scores.extend(list(enumerate(similarity_matrix[index])))

    # Sort the movies based on their similarity scores
    sorted_movies_by_similarity = sorted(all_similarity_scores, key=lambda x: x[1], reverse=True)

    # Select the top N movie recommendations, excluding the original input
    recommended_movie_indices = [i[0] for i in sorted_movies_by_similarity[1:top_n_recommendations + 1]]
    recommended_movies_list = movies.iloc[recommended_movie_indices]

    return recommended_movies_list[["title", "genre", "director", "rating"]]

# Main function to run the movie recommendation system
if __name__ == "__main__":
    print("Welcome to the Movie Recommendation System!")
    genre_input = input("Enter a genre to receive movie suggestions: ")
    movie_recommendations = suggest_movies_by_genre(genre_input)

    if isinstance(movie_recommendations, str):
        print(movie_recommendations)
    else:
        print("\nHere are some top movie recommendations for you:")
        print(movie_recommendations)


Welcome to the Movie Recommendation System!
Enter a genre to receive movie suggestions: Sci-Fi

Here are some top movie recommendations for you:
             title   genre           director  rating
2     Interstellar  Sci-Fi  Christopher Nolan     8.6
1       The Matrix  Sci-Fi     Lana Wachowski     8.7
0        Inception  Sci-Fi  Christopher Nolan     8.8
2     Interstellar  Sci-Fi  Christopher Nolan     8.6
3  The Dark Knight  Action  Christopher Nolan     9.0
